**Group Name: Team_Pycharmers**

**Team Members:**

        Akhil Surnedi (C0865688)

        Angelin Dafini (C0872335)

        Jiya Jha (C0861121)

        Swetha Sathiyakumar (C0862346)

**Model used- Gpt2**- The Reason for the selection of this model is, it has the ability to generate coherent and contextually relevant text for tasks such as language modeling, text completion, and question answering.

**Dataset used- SQUAD**- This dataset can then be used for various natural language processing tasks, including training and evaluating question answering models.


**Chatbot** -This chatbot is a medical chatbot designed to provide information and assistance related to diseases and symptoms. Whether you have health-related questions, want to learn about specific conditions, or need guidance on symptoms. Its also answering other domain related questions, but its some what related to the answer.

**Mandatory libraries installation before running the code**

In [ ]:
!pip install transformers[torch,accelerate]==4.30.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
pip install Datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


**Import the necessary libraries and modules**

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from tqdm import tqdm
from datasets import load_dataset
import random
from transformers import TrainingArguments,Trainer

**Load the SQuAD (Stanford Question Answering Dataset) dataset using the Hugging Face datasets library**

In [ ]:
squad_dataset = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

**Load GPT-2 model and tokenizer**

In [ ]:
config = GPT2Config.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", config=config)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

**Flatten the list of contexts**

In [ ]:
flat_contexts = [item for sublist in squad_dataset['train']['context'] for item in sublist]
num_examples_to_use = 20000
sampled_contexts = random.sample(flat_contexts, min(num_examples_to_use, len(flat_contexts)))

**Save the sampled contexts to a text file & Specify the path to the saved text file**

In [ ]:
with open("squad_train_sampled.txt", "w") as f:
    for context in sampled_contexts:
        f.write(context + "\n")
train_file_path = "squad_train_sampled.txt"

**Prepare training data**

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file_path,
    block_size=128,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


**Configure training arguments**

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2_squad",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=500
)

**Model Training Configuration**

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

**Train the model**

In [ ]:
trainer.train()
trainer.save_model("./gpt2_squad")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.612500


**Load GPT-2 model and tokenizer**

In [ ]:
config = GPT2Config.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", config=config)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

**Chatbot Interface - Type 'exit' to end the conversation**

In [ ]:
def chatbot_interface():
    print("Chatbot Interface - Type 'exit' to end the conversation")

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        input_ids = tokenizer.encode(user_input, return_tensors="pt")
        output = model.generate(
            input_ids,
            max_length=100,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.eos_token_id
        )
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f"Bot: {bot_response}")

if __name__ == "__main__":
    chatbot_interface()

Chatbot Interface - Type 'exit' to end the conversation
You: What are some coping strategies for stress and anxiety?
Bot: What are some coping strategies for stress and anxiety?

The following are a few of the most common coping techniques that can help you cope with stress.
...
 (1) Avoiding the negative. This is the first step in coping with anxiety. It's a common mistake to think that you're going to be able to cope without it. But, if you do, you'll be more likely to feel better. (2) Be aware of your emotions. If you feel
You: What are the benefits of regular physical activity?
Bot: What are the benefits of regular physical activity?

Regular physical exercise is a great way to get your body back to a healthy state. It's also a good way for your brain to recover from a bad day.
. Regular physical training is also an excellent way of getting your mind back into a normal state, which is why it's important to do regular exercise. You can also do it in a way that helps you focus on you